# <a id='toc1_'></a>[Model Evaluation](#toc0_)

This notebook contains the code to evaluate the models on the test set.

**Table of contents**<a id='toc0_'></a>    
- [Model Evaluation](#toc1_)    
- [Import libraries](#toc2_)    
- [Import data](#toc3_)    
- [Prepare data for training](#toc4_)    
  - [Neural network data](#toc4_1_)    
  - [Linear model data](#toc4_2_)    
- [Train Neural Network](#toc5_)    
  - [Graph the model](#toc5_1_)    
- [Train linear models](#toc6_)    
  - [OLS](#toc6_1_)    
  - [LASSO](#toc6_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Import libraries](#toc0_)

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.compose import ColumnTransformer
# import shap
import matplotlib.pyplot as plt
# from matplotlib.colors import TwoSlopeNorm
# import math
# import itertools
import warnings
from sklearn.exceptions import ConvergenceWarning

from libs.models import *
from libs.functions import *

plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 4), 'figure.dpi': 300})

%load_ext autoreload
%autoreload 2

# <a id='toc3_'></a>[Import data](#toc0_)

In [2]:
# read data
df = pd.read_csv('data/data.csv')

# <a id='toc4_'></a>[Prepare data for training](#toc0_)

In [ ]:
# prepare expanding window splits
periods = {
    '21' : '2020-01-01', # 2021 is the test set
    '22' : '2021-01-01', # 2022 is the test set
    '23' : '2022-01-01', # 2023 is the test set
    '24': '2023-01-01' # 2024 is the test set
}

# identify dummy vs. numeric columns
feature_cols = [col for col in df.columns if col not in ['timestamp', 'ticker', 'target']]
nace_cols = [c for c in feature_cols if c.startswith('NACE_')]
dummy_cols = ['divi','divo'] # sin removed
macro_cols = ['discount', 'tms', 'dp', 'ep', 'svar'] # 'bm_macro'

# nummeric cols = cols not in cat and macro cols
numeric_cols = [c for c in feature_cols if c not in dummy_cols and c not in nace_cols and c not in macro_cols]

# feature_cols = numeric_cols + dummy_cols + nace_cols # reorder columns to have numeric first

df_raw = df.copy(deep=True)
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])

# drop data from 2025
df_raw = df_raw[df_raw['timestamp'] < '2025-01-01']

In [ ]:
C = df[numeric_cols].values         # shape = (n_rows, P_c)
X = df[macro_cols].values           # shape = (n_rows, P_x)

# 1) compute all pairwise products with broadcasting:
#    this gives shape (n_rows, P_c, P_x)
K = C[:,:,None] * X[:,None,:]

# 2) reshape to (n_rows, P_c * P_x)
Z = K.reshape(len(df), -1)

# 3) build the column names in the same order
xc_names = [
    f"{c}_x_{m}"
    for c in numeric_cols
    for m in macro_cols
]

# 4) wrap back into a DataFrame
df_xc = pd.DataFrame(Z, columns=xc_names, index=df.index)

feature_cols = numeric_cols + xc_names + dummy_cols + nace_cols
numeric_cols = numeric_cols + xc_names
cat_cols = dummy_cols + nace_cols
df_z = df_raw.merge(df_xc, left_index=True, right_index=True)
# drop macro_cols
df_z = df_z.drop(columns=macro_cols)
# sort columns by feature_cols
df_norm = df_z[['timestamp', 'ticker', 'target'] + feature_cols]

y_values = df_norm['target'].values.astype('float32')


## <a id='toc4_1_'></a>[Neural network data](#toc0_)
Including a validation set

In [7]:
# prepare containers
X_train, X_val, X_test = {}, {}, {}
y_train, y_val, y_test = {}, {}, {}
preprocessors = {}

for y, period in periods.items():
    period = pd.to_datetime(period)

    # split masks
    tr_mask = df_norm['timestamp'] < period
    va_mask = (df_norm['timestamp'] >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=1) < period)
    te_mask = (df_norm['timestamp'] - pd.DateOffset(years=1) >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=2) < period)

    # extract raw feature DataFrames
    X_tr_df = df_norm.loc[tr_mask, feature_cols].copy()
    X_va_df = df_norm.loc[va_mask, feature_cols].copy()
    X_te_df = df_norm.loc[te_mask, feature_cols].copy()
    y_tr    = y_values[tr_mask]
    y_va    = y_values[va_mask]
    y_te    = y_values[te_mask]

    # compute winsorization bounds on train
    lower = X_tr_df[numeric_cols].quantile(0.01)
    upper = X_tr_df[numeric_cols].quantile(0.99)

    # apply clipping to train, val, test
    X_tr_df[numeric_cols] = X_tr_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_va_df[numeric_cols] = X_va_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_te_df[numeric_cols] = X_te_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)


    # now fit scaler on numeric only
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_cols),
        ('cat', 'passthrough',  cat_cols)
    ])
    preprocessor.fit(X_tr_df)
    preprocessors[y] = preprocessor

    # transform all splits
    X_train[y] = preprocessor.transform(X_tr_df).astype('float32')
    X_val[y]   = preprocessor.transform(X_va_df).astype('float32')
    X_test[y]  = preprocessor.transform(X_te_df).astype('float32')

    # store targets as before
    y_train[y] = y_tr.reshape(-1, 1)
    y_val[y]   = y_va.reshape(-1, 1)
    y_test[y]  = y_te.reshape(-1, 1)


## <a id='toc4_2_'></a>[Linear model data](#toc0_)
Excluding the validation set

In [8]:

Xlin_train, Xlin_test = {}, {}
ylin_train, ylin_test = {}, {}
preprocessors_lin = {}

cat_cols_lin = cat_cols + ['const']
feature_cols_lin = feature_cols + ['const']

for y, period in periods.items():
    period = pd.to_datetime(period)
    tr_mask = df_norm['timestamp']- pd.DateOffset(years=1) < period
    te_mask = (df_norm['timestamp'] - pd.DateOffset(years=1) >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=2) < period)

    # extract feature DataFrames
    X_tr_df = df_norm.loc[tr_mask, feature_cols]
    X_te_df = df_norm.loc[te_mask, feature_cols]
    y_tr = y_values[tr_mask]
    y_te = y_values[te_mask]

    # add constant column for linear regression
    X_tr_df['const'] = 1
    X_te_df['const'] = 1

    # compute winsorization bounds on train
    lower = X_tr_df[numeric_cols].quantile(0.01)
    upper = X_tr_df[numeric_cols].quantile(0.99)

    # apply clipping to train, test
    X_tr_df[numeric_cols] = X_tr_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_te_df[numeric_cols] = X_te_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)

    # fit scaler only on training set
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_cols),
        ('cat', 'passthrough', cat_cols_lin)
    ])
    preprocessor.fit(X_tr_df)
    preprocessors_lin[y] = preprocessor

    # ttransform splits
    Xlin_train[y] = preprocessor.transform(X_tr_df).astype('float32')
    Xlin_test[y]  = preprocessor.transform(X_te_df).astype('float32')

    # targets
    ylin_train[y] = y_tr
    ylin_test[y]  = y_te

# <a id='toc5_'></a>[Train Neural Network](#toc0_)

In [9]:
# moving to metal or CUDA GPU if available
device = torch.device(("cuda" if torch.cuda.is_available() 
                       else "mps" if torch.backends.mps.is_available() 
                       else "cpu"))
print("Using device:", device)

# general hyperparameters
hidden_depth = 3 # only hidden, excluding in- and output layers
hidden_width = 32 # int for all being equal width; list for different widths
learning_rate = 1e-4
activation_fun = nn.ReLU # nn.ReLU nn.Tanh nn.Sigmoid nn.LeakyReLU

# general critereon and regularization parameters
criterion = nn.MSELoss()
lambda_l1 = 1e-3 # l1 regularization
lambda_l2 = 1e-3 # l2 regularization
dropout = 0.3

# general parmeters
patience = 25 # 10% of epochs
print_freq = 250
epochs = 250
batch_size = 4096 # 8192 # 16384 # 2^1X adjust to your memory

n_runs = 5 # number of runs for each model to average over


Using device: mps


In [10]:
best_models = {}           # you can keep one “best” model if you like (e.g. the last run)
history   = {}
mlp_pred  = {}
models    = {}

for y, period in periods.items():
    print(f"\n=== Year {y}: running {n_runs} restarts ===")
    all_run_preds = []

    for run in range(n_runs):
        # set random seed for reproducibility (change for each run)
        seed = 42 + run
        np.random.seed(seed)
        torch.manual_seed(seed)
        print(f"Run {run+1}/{n_runs}, seed={seed}")

        # instantiate a fresh model & optimizer
        input_dim = X_train[y].shape[1]
        model = MLPModel(input_dim,
                         depth=hidden_depth,
                         width=hidden_width,
                         dropout=dropout,
                         activation=activation_fun).to(device)
        optimizer = torch.optim.Adam(model.parameters(),
                                      lr=learning_rate)

        # wrap datasets
        train_ds = MLPdataset(X_train[y], y_train[y])
        val_ds   = MLPdataset(X_val[y],   y_val[y])

        # train
        trained_model, hist = train_mlp(train_ds,
                                        val_ds,
                                        model,
                                        criterion,
                                        epochs,
                                        patience,
                                        print_freq,
                                        device,
                                        optimizer=optimizer,
                                        lambda_l1=lambda_l1,
                                        lambda_l2=lambda_l2,
                                        batch_size=batch_size,
                                        shuffle_train=True,
                                        shuffle_val=False,
                                        save_path=f'models/mlp_y{y}_l1{lambda_l1}_l2{lambda_l2}_drop{dropout}_lr{learning_rate}_w{hidden_width}_d{hidden_depth}_run{run+1}.pth'
                                        )

        # predict on test set
        preds = predict_mlp(trained_model,
                            X_test[y],
                            y_test=y_test[y],
                            scaler=preprocessors[y] if hasattr(preprocessors[y], 'inverse_transform') else None,
                            batch_size=batch_size,
                            device=device)
        all_run_preds.append(preds)

        # optionally store the last run’s model & history
        best_models[(y, run)] = trained_model
        history[(y, run)]     = hist

    # stack (n_runs, n_samples) → average over axis=0
    all_run_preds = np.stack(all_run_preds, axis=0)
    mlp_pred[y]   = np.mean(all_run_preds, axis=0)

    print(f"Averaged predictions for year {y} computed.")


=== Year 21: running 5 restarts ===
Run 1/5, seed=42
Early stopping at epoch 59
Best val loss: 1.96187E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.3_lr0.0001_w32_d3_run1.pth
Run 2/5, seed=43
Early stopping at epoch 186
Best val loss: 1.96973E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.3_lr0.0001_w32_d3_run2.pth
Run 3/5, seed=44
Early stopping at epoch 160
Best val loss: 1.97055E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.3_lr0.0001_w32_d3_run3.pth
Run 4/5, seed=45
Early stopping at epoch 87
Best val loss: 1.97042E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.3_lr0.0001_w32_d3_run4.pth
Run 5/5, seed=46
Early stopping at epoch 166
Best val loss: 1.97224E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.3_lr0.0001_w32_d3_run5.pth
Averaged predictions for year 21 computed.

=== Year 22: running 5 restarts ===
Run 1/5, seed=42
Early stopping at epoch 158
Best val loss: 1.22011E-02
Model saved to models/mlp_y22_l10.001_l20.001_drop0.3_lr0.

# <a id='toc6_'></a>[Train linear models](#toc0_)

## <a id='toc6_1_'></a>[OLS](#toc0_)

In [13]:
# linear model
# estimate the parameters
ols_est = {}
# ols_pred_train = {}
ols_pred = {}
ols_coeffs = {}


for y in periods.keys():
    print(f"Estimating OLS for {y}...")
    x_tr = Xlin_train[y]
    y_tr = ylin_train[y]
    x_te = Xlin_test[y]
    y_te = ylin_test[y]


    # estimate the parameters
    ols_est[y] = estimate(y_tr, x_tr)
    # ols_pred_train[y] = ols_est[y]['b_hat'] @ x_tr.T
    ols_pred[y] = ols_est[y]['b_hat'] @ x_te.T
    ols_coeffs[y] = ols_est[y]['b_hat']

Estimating OLS for 21...


/Users/johan/Documents/04 Uni/10 Thesis/git/MastersThesis/libs/functions.py:192: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov)).reshape(-1, 1)


Estimating OLS for 22...
Estimating OLS for 23...
Estimating OLS for 24...


In [14]:
ols_coeffs_df = pd.DataFrame(ols_coeffs, index=feature_cols_lin)
ols_coeffs_df = ols_coeffs_df.reindex(ols_coeffs_df.abs().sum(axis=1).sort_values(ascending=False).index, axis=0)
ols_coeffs_df = ols_coeffs_df.T
display(ols_coeffs_df)

,roic_x_tms,roaq_x_tms,roic_x_dp,roaq_x_dp,roic_x_svar,roaq_x_svar,roaq_x_discount,roic_x_discount,roaq_x_ep,roic_x_ep,...,cashpr_x_svar,lev_x_discount,ill_x_dp,turn_x_tms,pchgm_pchsale_x_discount,pctacc_x_dp,std_turn_x_discount,cashdebt_x_tms,salecash_x_dp,rd_sale_x_svar
21,-0.506179,0.511313,0.294592,-0.286156,0.797362,-0.797641,-0.134628,0.131583,-0.315503,0.407231,...,-0.000562,0.000103,-0.000107,0.000454,0.001054,0.000423,0.000326,-0.000426,-0.000517,-0.000433
22,-0.551004,0.548916,0.993486,-0.990598,-0.177860,0.178388,-0.177591,0.176125,-0.282070,0.318052,...,-0.001314,0.000694,-0.001100,0.000153,-0.000011,0.000763,0.000121,-0.000573,-0.000382,0.000059
23,1.764620,-1.768156,-2.346369,2.352144,-0.485175,0.482774,-0.219153,0.217865,0.051667,0.086960,...,-0.000564,0.001367,0.000375,-0.000221,0.000564,0.000505,0.000659,0.000201,0.000086,-0.000173
24,-2.107210,2.099086,0.302022,-0.298068,2.135682,-2.129545,2.916310,-2.908984,2.156285,-1.949453,...,-0.000469,0.000346,0.000848,-0.001581,0.000779,0.000264,-0.000595,0.000075,-0.000030,0.000285


## <a id='toc6_2_'></a>[LASSO](#toc0_)

In [15]:
# linear model
# create a grid using numpy.geomspace
penalty_grid = np.geomspace(1e-7, 100, num = 1000)
lasso_est = {}
# lasso_pred_train = {}
lasso_pred = {}
lasso_coeffs = {}


with warnings.catch_warnings():
    warnings.simplefilter("ignore", ConvergenceWarning)
    for y in periods.keys():
        print(f"Estimating LASSO for {y}...")
        x_tr = Xlin_train[y]
        y_tr = ylin_train[y]
        x_te = Xlin_test[y]
        y_te = ylin_test[y]

        # estimate the model using LassoCV
        fit_CV = LassoCV(cv=5, alphas=penalty_grid, max_iter=1000, eps=1e-3, n_jobs=-1).fit(x_tr,y_tr)
        # lasso_pred_train[y] = fit_CV.predict(x_tr)
        lasso_pred[y] = fit_CV.predict(x_te)

        # store the coefficients
        coeff = fit_CV.coef_
        lasso_coeffs[y] = coeff


Estimating LASSO for 21...
Estimating LASSO for 22...
Estimating LASSO for 23...
Estimating LASSO for 24...


In [16]:
lasso_coeffs_df = pd.DataFrame(lasso_coeffs, index=feature_cols_lin)
# sort by absolute value
lasso_coeffs_df = lasso_coeffs_df.reindex(lasso_coeffs_df.abs().sum(axis=1).sort_values(ascending=False).index, axis=0)

# drop columns with all zeros
lasso_coeffs_df = lasso_coeffs_df.T
lasso_coeffs_df = lasso_coeffs_df.loc[:, (lasso_coeffs_df != 0).any(axis=0)]
display(lasso_coeffs_df)

,indmom_x_discount,mom12m,age_x_discount,retvol_x_dp,age_x_dp,age_x_svar,indmom_x_svar,cfp_ia_x_discount,mom1m_x_discount,saleinv_x_svar,...,turn_x_discount,pchsale_pchinvt_x_dp,rd_x_tms,chatoia_x_svar,std_turn_x_tms,invest_x_svar,pricedelay_x_tms,acc_x_tms,mom12m_x_svar,dy_x_svar
21,0.011941,0.008003,-0.011049,0.004804,-0.005509,0.005513,-0.005822,0.003619,0.003745,0.003560,...,-0.000176,-0.000000,-0.000097,0.000151,-0.000046,-0.000115,0.000000,-0.00000,-0.000000,0.000000
22,0.011855,0.008319,-0.011579,0.005688,-0.004986,0.005956,-0.005752,0.003973,0.003462,0.003766,...,-0.000130,-0.000121,-0.000016,0.000077,-0.000045,-0.000000,0.000000,-0.00000,-0.000000,0.000000
23,0.010299,0.007802,-0.006855,0.006583,-0.003968,0.004533,-0.004049,0.004941,0.003819,0.003208,...,-0.000000,-0.000000,-0.000000,0.000000,-0.000136,-0.000000,0.000094,-0.00000,-0.000000,0.000004
24,0.009636,0.007448,-0.000604,0.006724,-0.005943,0.004270,-0.003508,0.005565,0.004731,0.003292,...,-0.000000,-0.000180,-0.000185,0.000000,-0.000000,-0.000000,0.000000,-0.00007,-0.000064,0.000000


# Summarize the results

In [17]:
pred_dfs = []

for y, period in periods.items():
    # rebuild masks
    te_mask = ((df_norm['timestamp'] - pd.DateOffset(years=1) >= period) &
               (df_norm['timestamp'] - pd.DateOffset(years=2) <  period))
    X_te_df = df_norm.loc[te_mask, feature_cols]
    idx = X_te_df.index


    pred_dfs.append(pd.DataFrame({
        'period':    y,
        'timestamp': df_norm.loc[idx, 'timestamp'],
        'ticker':    df_norm.loc[idx, 'ticker'],
        'y_true':    df_norm.loc[idx, 'target'].values.astype('float32'),
        'OLS':    ols_pred[y].flatten(),
        'LASSO':  lasso_pred[y],
        'MLP':    mlp_pred[y],
    }, index=idx))

all_preds = pd.concat(pred_dfs).sort_index()

In [18]:
methods = ['OLS', 
           'LASSO', 
           'MLP',
           ]

results = {}

for method in methods:
    rmse = rmse_fun(all_preds[method], all_preds['y_true'])
    mae = mae_fun(all_preds[method], all_preds['y_true'])
    results[method] = {
        'RMSE': rmse,
        'MAE': mae
    }

metrics = {}
for metric in ['RMSE','MAE']:
    key = f'{metric}'
    vals = [
        results['OLS'][metric],
        results['LASSO'][metric],
        results['MLP'][metric],
        ]
    metrics[key] = vals

tab = latex_table(methods,metrics)

print(tab)

\begin{tabular}{lccc}
\hline\hline \\ [-1.8ex]
 & OLS & LASSO & MLP \\ 
 \hline 
RMSE & 0.20040 & 0.19532 & 0.19463 \\ 
MAE & 0.08743 & 0.07925 & 0.07743 \\ 
\hline\hline
\end{tabular}


In [19]:
metrics_full = {}
results_full = {}
for y, period in periods.items():
    for method in methods:
        y_true = all_preds.loc[all_preds['period'] == y, 'y_true']
        y_pred = all_preds.loc[all_preds['period'] == y, method]
        key = f'{method}{y}'
        results_full[key] = {
            'RMSE': rmse_fun(y_pred, y_true),
            'MAE': mae_fun(y_pred, y_true)
        }
    
    for metric in ['RMSE','MAE']:
        key = f'*{metric}*20{y}'
        vals = [
            results_full[f'OLS{y}'][metric],
            results_full[f'LASSO{y}'][metric],
            results_full[f'MLP{y}'][metric],
        ]
        metrics_full[key] = vals

tab_full = latex_table_grouped(methods,metrics_full)

print(tab_full)


\begin{tabular}{clccc}
\hline\hline \\ [-1.8ex]
 &  & OLS & LASSO & MLP \\ 
 \hline 
\multirow[c]{4}{*}{\rotatebox{90}{RMSE}}& 2021 & 0.11697 & 0.10895 & 0.10968 \\ 
 & 2022 & 0.32097 & 0.31817 & 0.31748 \\ 
 & 2023 & 0.12683 & 0.11483 & 0.11352 \\ 
 & 2024 & 0.16596 & 0.16129 & 0.15963 \\ 
\hline\multirow[c]{4}{*}{\rotatebox{90}{MAE}}& 2021 & 0.08267 & 0.07238 & 0.07204 \\ 
 & 2022 & 0.10512 & 0.10074 & 0.09741 \\ 
 & 2023 & 0.08146 & 0.06964 & 0.06835 \\ 
 & 2024 & 0.08019 & 0.07404 & 0.07164 \\ 
\hline\hline
\end{tabular}
